In [ ]:
from GridOperation import Integration
from spatiallyAdaptiveSingleDimension2 import SpatiallyAdaptiveSingleDimensions2
%matplotlib inline

from PerformTestCase import *
from Function import *
from ErrorCalculator import *

# evaluation points for error estimation
def evaluation_points(a, b, d):
    grid = np.meshgrid(*(np.linspace(a[i], b[i], 10) for i in range(len(a))))
    points = list(zip(*(x.flat for x in grid)))

    return points


# shortcut for romberg SASD
def get_single_dim_romberg(a, b, f, reference_solution, sasd_version,
                           slice_grouping, slice_version, container_version,
                           force_balanced_refinement_tree):
    grid_romberg = GlobalRombergGrid(a=a, b=b, modified_basis=False, boundary=True,
                                     slice_grouping=slice_grouping,
                                     slice_version=slice_version,
                                     container_version=container_version)

    operation_romberg = Integration(f=f, grid=grid_romberg, dim=dim, reference_solution=reference_solution)

    return SpatiallyAdaptiveSingleDimensions2(a, b, operation=operation_romberg, rebalancing=False, version=sasd_version,
                                              force_balanced_refinement_tree=force_balanced_refinement_tree)

# Settings
dim = 3
a = np.zeros(dim)
b = np.ones(dim)
max_tol = 20  # set to max, for better comparison
max_evaluations = 10 ** 4
evaluation_points = evaluation_points(a, b, dim)
functions = []
sasd_version = 3

# --- Smooth functions

# GenzCornerPeak: Coefficients from SGA Split Extend Paper, p. 18
coeffs = np.array([np.float64(3 * i) for i in range(1, dim + 1)])
f_genz_corner = GenzCornerPeak(coeffs=coeffs)

# GenzProductPeak: Coefficients from SGA Split Extend Paper, p. 18
coeffs = np.array([np.float64(3 * i) for i in range(1, dim + 1)])
midpoint = np.ones(dim) * 0.99
f_genz_product = GenzProductPeak(coeffs, midpoint)

# GenzContinious: Coefficients from SGA Split Extend Paper, p. 18
coeffs = np.array([np.float64(3 * i) for i in range(1, dim + 1)])
midpoint = np.ones(dim) * 0.5
f_genz_cont = GenzC0(coeffs, midpoint)

# GenzGaussian: Coefficients from SGA Split Extend Paper, p. 18
coeffs = np.array([np.float64(i) for i in range(1, dim + 1)])
midpoint = np.ones(dim) * 0.99
f_genz_gaussian = GenzGaussian(coeffs, midpoint)

# FunctionExpVar: See SGA Split Extend Paper, p. 18
f_exp_var = FunctionExpVar()

# GenzOszillatory: https://www.sfu.ca/~ssurjano/oscil.html
coeffs = np.array([np.float64(i) for i in range(1, dim + 1)])
offset = 0.5
f_genz_osz = GenzOszillatory(coeffs, offset)

# --- Discontinious functions

# GenzContinious: Coefficients from SGA Split Extend Paper, p. 18
border = np.ones(dim) * 0.2
coeffs = np.array([np.float64(3 * i) for i in range(1, dim + 1)])
f_genz_disc = GenzDiscontinious(border=border,coeffs=coeffs)

# Manually override functions array:
functions = [
    f_genz_corner,
    f_genz_product,
    # f_genz_cont,
    # f_genz_gaussian,
    # f_exp_var,
    # f_genz_osz,
    # f_genz_disc
]

# Iterate over all functions
for f in functions:
    # Plot
    f.plot(np.ones(dim)*a,np.ones(dim)*b,
           # filename=str(f.__class__.__name__)
           )
    reference_solution = f.getAnalyticSolutionIntegral(a,b)

    # Trapezoidal Grid
    grid_trapezoidal = GlobalTrapezoidalGrid(a=a, b=b, modified_basis=False, boundary=True)
    operation_trapezoidal = Integration(f=f, grid=grid_trapezoidal, dim=dim, reference_solution=reference_solution)
    trapezoidal = SpatiallyAdaptiveSingleDimensions2(a, b, operation=operation_trapezoidal, rebalancing=False,
                                                     version=sasd_version,
                                                     force_balanced_refinement_tree=False)
    
    operation_trapezoidal = Integration(f=f, grid=grid_trapezoidal, dim=dim, reference_solution=reference_solution)
    trapezoidal_rebalancing = SpatiallyAdaptiveSingleDimensions2(a, b, operation=operation_trapezoidal, rebalancing=True,
                                                                 version=sasd_version,
                                                                 force_balanced_refinement_tree=False)
    
    # grids for standard combi
    grid = TrapezoidalGrid(a, b, boundary=True)
    grid_gauss = GaussLegendreGrid(a=a, b=b)

    # Unit Slices
    r_unit = get_single_dim_romberg(a, b, f, reference_solution,
                                    sasd_version=sasd_version,
                                    slice_grouping=SliceGrouping.UNIT,
                                    slice_version=SliceVersion.ROMBERG_DEFAULT,
                                    container_version=SliceContainerVersion.ROMBERG_DEFAULT,
                                    force_balanced_refinement_tree=False)

    r_full_unit = get_single_dim_romberg(a, b, f, reference_solution,
                                         sasd_version=sasd_version,
                                         slice_grouping=SliceGrouping.UNIT,
                                         slice_version=SliceVersion.ROMBERG_DEFAULT,
                                         container_version=SliceContainerVersion.ROMBERG_DEFAULT,
                                         force_balanced_refinement_tree=True)

    r_trapezoidal = get_single_dim_romberg(a, b, f, reference_solution,
                                           sasd_version=sasd_version,
                                           slice_grouping=SliceGrouping.UNIT,
                                           slice_version=SliceVersion.TRAPEZOID,
                                           container_version=SliceContainerVersion.ROMBERG_DEFAULT,
                                           force_balanced_refinement_tree=False)

    # Slice Grouping
    r_grouped = get_single_dim_romberg(a, b, f, reference_solution,
                                       sasd_version=sasd_version,
                                       slice_grouping=SliceGrouping.GROUPED,
                                       slice_version=SliceVersion.ROMBERG_DEFAULT,
                                       container_version=SliceContainerVersion.ROMBERG_DEFAULT,
                                       force_balanced_refinement_tree=False)

    r_full_grouped = get_single_dim_romberg(a, b, f, reference_solution,
                                            sasd_version=sasd_version,
                                            slice_grouping=SliceGrouping.GROUPED,
                                            slice_version=SliceVersion.ROMBERG_DEFAULT,
                                            container_version=SliceContainerVersion.ROMBERG_DEFAULT,
                                            force_balanced_refinement_tree=True)
    
    r_grouped_trapezoidal = get_single_dim_romberg(a, b, f, reference_solution,
                                                   sasd_version=sasd_version,
                                                   slice_grouping=SliceGrouping.GROUPED,
                                                   slice_version=SliceVersion.TRAPEZOID,
                                                   container_version=SliceContainerVersion.ROMBERG_DEFAULT,
                                                   force_balanced_refinement_tree=False)
    
    r_full_grouped_trapezoidal = get_single_dim_romberg(a, b, f, reference_solution,
                                                        sasd_version=sasd_version,
                                                        slice_grouping=SliceGrouping.GROUPED,
                                                        slice_version=SliceVersion.TRAPEZOID,
                                                        container_version=SliceContainerVersion.ROMBERG_DEFAULT,
                                                        force_balanced_refinement_tree=True)

    # Optimized slice grouping
    r_grouped_optimized = get_single_dim_romberg(a, b, f, reference_solution,
                                                 sasd_version=sasd_version,
                                                 slice_grouping=SliceGrouping.GROUPED_OPTIMIZED,
                                                 slice_version=SliceVersion.ROMBERG_DEFAULT,
                                                 container_version=SliceContainerVersion.ROMBERG_DEFAULT,
                                                 force_balanced_refinement_tree=False)

    r_full_grouped_optimized = get_single_dim_romberg(a, b, f, reference_solution,
                                                      sasd_version=sasd_version,
                                                      slice_grouping=SliceGrouping.GROUPED_OPTIMIZED,
                                                      slice_version=SliceVersion.ROMBERG_DEFAULT,
                                                      container_version=SliceContainerVersion.ROMBERG_DEFAULT,
                                                      force_balanced_refinement_tree=True)

    # Unit Slices with sliced Romberg
    r_unit_sliced = get_single_dim_romberg(a, b, f, reference_solution,
                                           sasd_version=sasd_version,
                                           slice_grouping=SliceGrouping.UNIT,
                                           slice_version=SliceVersion.ROMBERG_SLICED,
                                           container_version=SliceContainerVersion.ROMBERG_DEFAULT,
                                           force_balanced_refinement_tree=False)

    r_full_unit_sliced = get_single_dim_romberg(a, b, f, reference_solution,
                                                sasd_version=sasd_version,
                                                slice_grouping=SliceGrouping.UNIT,
                                                slice_version=SliceVersion.ROMBERG_SLICED,
                                                container_version=SliceContainerVersion.ROMBERG_DEFAULT,
                                                force_balanced_refinement_tree=True)
    
    
    # Interpolating Romberg
    r_interpolating = get_single_dim_romberg(a, b, f, reference_solution,
                                             sasd_version=sasd_version,
                                             slice_grouping=SliceGrouping.GROUPED_OPTIMIZED,
                                             slice_version=SliceVersion.ROMBERG_SLICED,
                                             container_version=SliceContainerVersion.LAGRANGE_ROMBERG,
                                             force_balanced_refinement_tree=False)

    # Variants that are compared
    errorOperator = ErrorCalculatorSingleDimVolumeGuided()

    algorithmArray = [
        (trapezoidal, 1, 2, errorOperator, 'Trapezoidal Grid'),
        (trapezoidal_rebalancing, 1, 2, errorOperator, 'Trapezoidal Grid (Rebalancing)'),

        # (r_trapezoidal, 1, 2, errorOperator, 'Romberg Grid (Unit, Trapezoid)'),

        (r_unit, 1, 2, errorOperator, 'Romberg Grid (Unit, Default)'),
        (r_full_unit, 1, 2, errorOperator, 'Romberg Grid (Full, Unit, Default)'),

        # (r_unit_sliced, 1, 2, errorOperator, 'Romberg Grid (Unit, Sliced)'),
        # (r_full_unit_sliced, 1, 2, errorOperator, 'Romberg Grid (Full, Unit, Sliced)'),

        (r_grouped, 1, 2, errorOperator, 'Romberg Grid (Grouped, Default)'),
        (r_full_grouped, 1, 2, errorOperator, 'Romberg Grid (Full, Grouped, Default)'),

        (r_grouped_trapezoidal, 1, 2, errorOperator, 'Romberg Grid (Grouped, Trapezoid)'),
        (r_full_grouped_trapezoidal, 1, 2, errorOperator, 'Romberg Grid (Full, Grouped, Trapezoid)'),

        (r_grouped_optimized, 1, 2, errorOperator, 'RombergGrid (Grouped Optimized, Default)'),
        (r_full_grouped_optimized, 1, 2, errorOperator, 'RombergGrid (Full, Grouped Optimized, Default)'),

        (r_interpolating, 1, 2, errorOperator, 'InterpolatingRombergGrid (Grouped Optimized, Default)'),        
    ]

    function_name = f.__class__.__name__
    filename = "error_comparison_{}_{}d".format(function_name, dim)

    performTestcaseArbitraryDim(f, a, b, algorithmArray,
                                max_tol, dim, 6, grid=grid_gauss, evaluation_points=evaluation_points,
                                max_evaluations=max_evaluations,
                                calc_standard_schemes=True, # enable for standard schemes
                                minLmin=1,
                                maxLmin=3,
                                standard_combi_grid_name="Gauss-Legendre Grid",
                                legend_title="{} function".format(function_name),
                                filename=filename,
                                save_csv=True,
                                save_plot=True
                                )